In [2]:
!git clone https://github.com/shivigup/EE603-Machine-Learning-for-Signal-Processing
%cd EE603-Machine-Learning-for-Signal-Processing/project

Cloning into 'EE603-Machine-Learning-for-Signal-Processing'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (354/354), done.
remote: Compressing objects: 100% (340/340), done.
remote: Total 354 (delta 107), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (354/354), 449.99 MiB | 18.06 MiB/s, done.
Resolving deltas: 100% (107/107), done.
Checking out files: 100% (192/192), done.
/content/EE603-Machine-Learning-for-Signal-Processing/project


In [3]:
import glob
import numpy as np
import pandas as pd
import librosa
import random as rd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [4]:
def load_spectogram(dirname):
  i=0;
  filename= 'temp'
  for name in glob.glob(dirname+'/*.npy'):
    spectog = np.load(name)
    name = name[12:len(name)-4]
    if(i==0):
      spec= np.empty((0,spectog.shape[0],spectog.shape[1]))
      filename = name
      i=1
    elif(i==1):
      filename = np.vstack([filename,name])
    spectog=spectog.reshape((1,spectog.shape[0],spectog.shape[1]))
    spec=np.vstack([spec,spectog])
  #print(filename)
  return filename,spec

In [5]:
filename,spec = load_spectogram('spectrogram')
model = keras.models.load_model('Framewise_NN')

In [6]:
predictions = np.empty((0,313))
for i in range(spec.shape[0]):
  pred_index = np.zeros((313))
  pred = model.predict(np.transpose(spec[i,:]))
  for j in range(313):
    pred_index[j] = np.argmax(pred[j,:])
  pred_index=pred_index.reshape((1,313))
  predictions=np.vstack([predictions,pred_index])

In [7]:
for i in range(predictions.shape[0]):
  for j in range(1, predictions.shape[1]-1):
    if(predictions[i, j-1]==predictions[i, j+1]):
        predictions[i, j] = predictions[i,j-1]

In [8]:
label_names = ['music', 'speech']

In [15]:
df = pd.DataFrame(columns=['filename', 'onset', 'offset', 'label'])
for i in range(predictions.shape[0]):
  onset = 0
  offset = 0
  state = 0
  for j in range(1, predictions.shape[1]):
    if(state==0):
      if(predictions[i,j]!=0 and predictions[i, j-1]==0):
        onset = (512*(j-1)+1024)/16000
        state = predictions[i, j]
    else:
      if(predictions[i,j]==0 and predictions[i, j-1]!=0):
        offset = (512*(j-2)+1024)/16000
        if(offset-onset>0.2):
          df = df.append({'filename': filename[i][0], 'onset' : onset, 'offset' : offset, 'label' : label_names[int(state-1)]}, ignore_index=True)
          #print(filename[i,0], ' ', onset, ' ', offset, ' ', int(state-1))
        state = 0

In [16]:
df

,filename,onset,offset,label
0,music_noisy2,0.576,2.848,music
1,music_noisy2,2.944,4.288,music
2,music_noisy2,4.640,5.184,music
3,music_noisy2,5.408,5.824,music
4,music_noisy2,6.048,8.768,music
...,...,...,...,...
88,music_noisy5,0.544,2.976,music
89,music_noisy5,3.424,4.640,music
90,music_noisy5,4.992,5.248,music
91,music_noisy5,6.272,9.472,music


In [17]:
df.to_csv('framewise_NN.csv', index=False)